# Capstone Project - The Battle of Neighbourhood
# Neighbourhood recommendation system for immigrants planning to move to Toronto

### Installing and Importing python libraries

In [2]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 98 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 4.6 MB/s  eta 0:00:01


In [3]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Data Wrangling

Importing data from wikipedia page to a Data Frame.

Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [9]:
data_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.read_html(data_page.content, header=0)[0]

In [10]:
df = df[df['Borough'] != 'Not assigned']

In [13]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
df.shape

(103, 3)

Import Latitude and Longitude values from file

In [15]:
coordinates_df = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_toronto = pd.merge(df, coordinates_df, on = 'Postal Code')
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Toronto coordinates were used to set the center location:

In [19]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


Mapping the neighbourhoods based on their coordinates (latitude and longitude).

In [22]:
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods

Define Foursquare credentials and version

In [23]:
CLIENT_ID = '5O3XNCTGA1HG5H442L15ZOYHLBBPYDFT5ZUZYYAKIOC5NHVH' # your Foursquare ID
CLIENT_SECRET = 'ADMQYLOHUDOD5DRUOK1VIKNZAD5QI0HKAM0S3USEEYKUEVCD' # your Foursquare Secret
VERSION = '20190326' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5O3XNCTGA1HG5H442L15ZOYHLBBPYDFT5ZUZYYAKIOC5NHVH
CLIENT_SECRET:ADMQYLOHUDOD5DRUOK1VIKNZAD5QI0HKAM0S3USEEYKUEVCD


Create a function which will return near by venues

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The function is run for all neighbourhoods in Toronto and a new dataframe called Toronto_venues is created

In [25]:
Toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                 )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Number of venues created for each neighbourhood

In [26]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


Number of unique categories found from the generated data.

In [27]:
len(Toronto_venues['Venue Category'].unique())

273

### Neighborhood venues analysis

Onehot coding is used to create a dataframe with occurence of each venue present (1) or not (0) under each Neighborhood

In [36]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']],prefix="",prefix_sep="")

Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 


fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

The rows are grouped by neighborhood and the mean of the frequency of occurrence of each category is calculated.

In [38]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0

Set Neighbourhood as index.

In [41]:
Toronto_grouped.set_index('Neighbourhood', inplace=True)
Toronto_grouped.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [42]:
Toronto_grouped.shape

(94, 272)

### Feature Selection
Currently there are 272 venue categories in the dataset with a lot of them belonging to the same general "group". i.e there is "Vietnamese Restaurant", "Greek Restaurant", "Vegetarian Restaurant" all belonging to the same logical group. The 272 venues will be filtered through a key words search so to create a number of higher level categories which will be used for both clustering as well as recommendation.

For this purpose the categories created with corresponding filters are:

1. Restaurants - keywords (Restaurant|Steak|Bistro)
2. Fast_Food - keywords (Fast|Burger|Food Truck|Sandwich|Pizza Place|Place|Joint)
3. Health_Fitness - keywords (Yoga|Gym|Martial|Health|Fitness|Pool|Healthy|Field|Sports|Trail|Stadium|Tennis|Climbing)
4. Kids_Friendly - keywords (Kid|Play|Park|Nursery|Playground|School|College|Nursery|Medical Center|Clinic|Basketball|Football)
5. Nightlife' - keywords (Night|Bar|Pub|Wine|Dance|Strip Club|Adult|Club|Speakeasy|Liquor|Brewery)
6. Culture - keywords (Gallery|Museum|Paintings|Monument|Church|Theater|Cinema|Sculpture|Aquarium|Concert|Art)
7. Shopping - keywords (Mall|Shopping|Shop|Retail|Store|Boutique)
8. Short_stay - keywords (Motel|Hotel|Airport|Stay)
9. Personal_care - keywords (Salon|Barber|Spa|Massage|Hair|Nail|Tanning)
10. Food_markets - keywords (Supermarket|Market|Grocery|Bakery|Butcher)
11. Transportation - keywords (Train|Metro|Bus|Tram|Boat)
12. Leisure - keywords (Beach|Plaza|Skating)

In [43]:
pd.options.mode.chained_assignment = None

Toronto_1=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Restaurant|Steak|Bistro')]
Toronto_1['total1']=Toronto_1.sum(axis=1)

Toronto_2=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Fast|Burger|Food Truck|Sandwich|Pizza Place|Place|Joint')]
Toronto_2['total2']=Toronto_2.sum(axis=1)

Toronto_3=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Yoga|Gym|Martial|Health|Fitness|Pool|Healthy|Field|Sports|Trail|Stadium|Tennis|Climbing')]
Toronto_3['total3']=Toronto_3.sum(axis=1)

Toronto_4=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Kid|Play|Park|Nursery|Playground|School|College|Nursery|Medical Center|Clinic|Basketball|Football')]
Toronto_4['total4']=Toronto_4.sum(axis=1)

Toronto_5=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Night|Bar|Pub|Wine|Dance|Strip Club|Adult|Club|Speakeasy|Liquor|Brewery')]
Toronto_5['total5']=Toronto_5.sum(axis=1)

Toronto_6=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Gallery|Museum|Paintings|Monument|Church|Theater|Cinema|Sculpture|Aquarium|Concert|Art')]
Toronto_6['total6']=Toronto_6.sum(axis=1)

Toronto_7=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Mall|Shopping|Shop|Retail|Store|Boutique')]
Toronto_7['total7']=Toronto_7.sum(axis=1)

Toronto_8=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Motel|Hotel|Airport|Stay')]
Toronto_8['total8']=Toronto_8.sum(axis=1)

Toronto_9=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Salon|Barber|Spa|Massage|Hair|Nail|Tanning')]
Toronto_9['total9']=Toronto_9.sum(axis=1)

Toronto_10=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Supermarket|Market|Grocery|Bakery|Butcher')]
Toronto_10['total10']=Toronto_10.sum(axis=1)

Toronto_11=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Train|Metro|Bus|Tram|Boat')]
Toronto_11['total11']=Toronto_11.sum(axis=1)

Toronto_12=Toronto_grouped.loc[:, Toronto_grouped.columns.str.contains('Beach|Plaza|Skating')]
Toronto_12['total12']=Toronto_12.sum(axis=1)

In [44]:
Toronto_grouped['Restaurants']=Toronto_1['total1']
Toronto_grouped['Fast_Food']=Toronto_2['total2']
Toronto_grouped['Health_Fitness']=Toronto_3['total3']
Toronto_grouped['Kids_Friendly']=Toronto_4['total4']
Toronto_grouped['Nightlife']=Toronto_5['total5']
Toronto_grouped['Culture']=Toronto_6['total6']
Toronto_grouped['Shopping']=Toronto_7['total7']
Toronto_grouped['Short_stay']=Toronto_8['total8']
Toronto_grouped['Personal_care']=Toronto_9['total9']
Toronto_grouped['Food_markets']=Toronto_10['total10']
Toronto_grouped['Transportation']=Toronto_11['total11']
Toronto_grouped['Leisure']=Toronto_12['total12']
Toronto_grouped.head(5)

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Restaurants,Fast_Food,Health_Fitness,Kids_Friendly,Nightlife,Culture,Shopping,Short_stay,Personal_care,Food_markets,Transportation,Leisure
Nei

In [45]:
Toronto_grouped_new= Toronto_grouped[['Restaurants',
                                    'Fast_Food',
                                    'Health_Fitness',
                                    'Kids_Friendly',
                                    'Nightlife',
                                    'Culture',
                                    'Shopping',
                                    'Short_stay',
                                    'Personal_care',
                                    'Food_markets',
                                    'Transportation',
                                    'Leisure']].copy()
Toronto_grouped_new.head()

,Restaurants,Fast_Food,Health_Fitness,Kids_Friendly,Nightlife,Culture,Shopping,Short_stay,Personal_care,Food_markets,Transportation,Leisure
Neighbourhood,,,,,,,,,,,,
Agincourt,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.25
"Alderwood, Long Branch",0.000000,0.428571,0.142857,0.0,0.142857,0.0,0.142857,0.0,0.0,0.000000,0.0,0.00
"Bathurst Manor, Wilson Heights, Downsview North",0.142857,0.142857,0.000000,0.0,0.000000,0.0,0.380952,0.0,0.0,0.095238,0.0,0.00
Bayview Village,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00
"Bedford Park, Lawrence Manor East",0.391304,0.130435,0.000000,0.0,0.130435,0.0,0.260870,0.0,0.0,0.086957,0.0,0.00


In [46]:
Toronto_grouped_new.shape

(94, 12)

Let's print each neighborhood along with the top 5 most common venues
First the Neighborhood is returned as a column in the dataframe

In [47]:
Toronto_grouped_new.reset_index(level=0,inplace=True)

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 5

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped_new['Neighbourhood']

for ind in np.arange(Toronto_grouped_new.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped_new.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(10)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Leisure,Restaurants,Transportation,Food_markets,Personal_care
1,"Alderwood, Long Branch",Fast_Food,Shopping,Nightlife,Health_Fitness,Leisure
2,"Bathurst Manor, Wilson Heights, Downsview North",Shopping,Fast_Food,Restaurants,Food_markets,Leisure
3,Bayview Village,Restaurants,Leisure,Transportation,Food_markets,Personal_care
4,"Bedford Park, Lawrence Manor East",Restaurants,Shopping,Nightlife,Fast_Food,Food_markets
5,Berczy Park,Shopping,Restaurants,Nightlife,Food_markets,Culture
6,"Birch Cliff, Cliffside West",Leisure,Kids_Friendly,Health_Fitness,Transportation,Food_markets
7,"Brockton, Parkdale Village, Exhibition Place",Shopping,Health_Fitness,Food_markets,Culture,Nightlife
8,"Business reply mail Processing Centre, South C...",Fast_Food,Food_markets,Kids_Friendly,Restaurants,Personal_care
9,"CN Tower, King and Spadina, Railway Lands, Har...",Short_stay,Shopping,Transportation,Culture,Nightlife


In [55]:
neighbourhoods_venues_sorted.to_csv("top5.csv", index=False)

### K-means Clustering

Preprocessing the data.

In [56]:
from sklearn.preprocessing import StandardScaler
X = Toronto_grouped_new.values[:,1:].astype(float)
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 0.39487793, -0.74915845, -0.52822975, ..., -0.63863091,
        -0.24607752,  5.32995767],
       [-1.05411213,  2.60901663,  0.40111789, ..., -0.63863091,
        -0.24607752, -0.26010574],
       [-0.22611781,  0.37023325, -0.52822975, ...,  0.54511916,
        -0.24607752, -0.26010574],
       ...,
       [ 0.87787462, -0.74915845, -0.52822975, ..., -0.63863091,
        -0.24607752, -0.26010574],
       [-1.05411213, -0.74915845,  0.40111789, ..., -0.63863091,
         1.64822465,  2.93421621],
       [-1.05411213, -0.74915845, -0.52822975, ..., -0.63863091,
        -0.24607752, -0.26010574]])

In [57]:
from sklearn.cluster import KMeans
num_clusters = 5
k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[4 4 4 4 4 1 3 1 4 0 3 4 4 4 4 1 4 0 1 4 4 4 4 1 4 1 3 4 4 1 2 4 1 2 4 1 1
 3 3 3 3 1 4 1 2 4 2 4 1 4 1 4 3 4 3 4 1 4 3 1 4 3 4 1 1 3 4 1 4 4 3 4 1 4
 4 1 4 4 1 1 4 4 4 1 1 4 4 4 4 4 4 4 2 3]


In [58]:
Toronto_grouped_new["Labels"] = labels
Toronto_grouped_new.head()

,Neighbourhood,Restaurants,Fast_Food,Health_Fitness,Kids_Friendly,Nightlife,Culture,Shopping,Short_stay,Personal_care,Food_markets,Transportation,Leisure,Labels
0,Agincourt,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.25,4
1,"Alderwood, Long Branch",0.000000,0.428571,0.142857,0.0,0.142857,0.0,0.142857,0.0,0.0,0.000000,0.0,0.00,4
2,"Bathurst Manor, Wilson Heights, Downsview North",0.142857,0.142857,0.000000,0.0,0.000000,0.0,0.380952,0.0,0.0,0.095238,0.0,0.00,4
3,Bayview Village,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,4
4,"Bedford Park, Lawrence Manor East",0.391304,0.130435,0.000000,0.0,0.130435,0.0,0.260870,0.0,0.0,0.086957,0.0,0.00,4


Merge data frames Toronto_grouped_new and df_toronto

In [59]:
toronto_merged = Toronto_grouped_new
toronto_merged = toronto_merged.join(df_toronto.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head()

,Neighbourhood,Restaurants,Fast_Food,Health_Fitness,Kids_Friendly,Nightlife,Culture,Shopping,Short_stay,Personal_care,Food_markets,Transportation,Leisure,Labels,Postal Code,Borough,Latitude,Longitude
0,Agincourt,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.25,4,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",0.000000,0.428571,0.142857,0.0,0.142857,0.0,0.142857,0.0,0.0,0.000000,0.0,0.00,4,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",0.142857,0.142857,0.000000,0.0,0.000000,0.0,0.380952,0.0,0.0,0.095238,0.0,0.00,4,M3H,North York,43.754328,-79.442259
3,Bayview Village,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,4,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",0.391304,0.130435,0.000000,0.0,0.130435,0.0,0.260870,0.0,0.0,0.086957,0.0,0.00,4,M5M,North York,43.733283,-79.419750


In [60]:
toronto_merged.reset_index(level=0, inplace=True)

In [80]:
toronto_merged.to_csv("clusters.csv", index=False)

,index,Neighbourhood,Restaurants,Fast_Food,Health_Fitness,Kids_Friendly,Nightlife,Culture,Shopping,Short_stay,Personal_care,Food_markets,Transportation,Leisure,Labels,Postal Code,Borough,Latitude,Longitude
0,0,Agincourt,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,4,M1S,Scarborough,43.794200,-79.262029
1,1,"Alderwood, Long Branch",0.000000,0.428571,0.142857,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,4,M8W,Etobicoke,43.602414,-79.543484
2,2,"Bathurst Manor, Wilson Heights, Downsview North",0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.380952,0.000000,0.000000,0.095238,0.000000,0.000000,4,M3H,North York,43.754328,-79.442259
3,3,Bayview Village,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4,M2K,North York,43.786947,-79.385975
4,4,"Bedford Park, Lawrence Manor East",0.391304,0.130435,0.000000,0.000000,0.130435,0.000000,0.260870,0.000000,0.000000,0.086957,0.000000,0.000000,4,M5M,North York,43.733283,-79.419750
5,5,Berczy Park,0.245614,0.000000,0.017544,0.035088,0.175439,0.052632,0.245614,0.017544,0.000000,0.122807,0.000000,0.017544,1,M5E,Downtown Toronto,43.644771,-79.373306
6,6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,3,M1N,Scarborough,43.692657,-79.264848
7,7,"Brockton, Parkdale Village, Exhibition Place",0.080000,0.040000,0.160000,0.000000,0.080000,0.080000,0.240000,0.000000,0.000000,0.120000,0.000000,0.000000,1,M6K,West Toronto,43.636847,-79.428191
8,8,"Business reply mail Processing Centre, South C...",0.125000,0.187500,0.062500,0.125000,0.062500,0.000000,0.062500,0.000000,0.062500,0.125000,0.000000,0.000000,4,M7Y,East Toronto,43.662744,-79.321558
9,9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.117647,0.529412,0.000000,0.000000,0.058824,0.000000,0,M5V,Downtown Toronto,43.628947,-79.394420


In [66]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

Let's feed the system with user ratings for each of the 12 categories we created depending on how important each was for the user in a scale of 1-10. For the testing purpose we used a “persona” of a user that is married with children and rated each category accordingly.

In [68]:
userInput_Family = [
            {'Venue':'Restaurants','Rating':7},
            {'Venue':'Fast_Food','Rating':5},
            {'Venue':'Health_Fitness','Rating':8},
            {'Venue':'Kids_Friendly','Rating':10},
            {'Venue':'Nightlife','Rating':1},
            {'Venue':'Culture','Rating':8},
            {'Venue':'Shopping','Rating':9},
            {'Venue':'Short_stay','Rating':0},
            {'Venue':'Personal_care','Rating':6},
            {'Venue':'Food_markets','Rating':10},
            {'Venue':'Transportation','Rating':5},
            {'Venue':'Leisure','Rating':8}
         ] 
inputVenues_Family = pd.DataFrame(userInput_Family) 

inputVenues_Family

,Venue,Rating
0,Restaurants,7
1,Fast_Food,5
2,Health_Fitness,8
3,Kids_Friendly,10
4,Nightlife,1
5,Culture,8
6,Shopping,9
7,Short_stay,0
8,Personal_care,6
9,Food_markets,10


In [69]:
inputVenues_Family.set_index('Venue', inplace=True)

In [71]:
Toronto_grouped_new.set_index('Neighbourhood',inplace=True)

In [72]:
Toronto_grouped_new.head()

,Restaurants,Fast_Food,Health_Fitness,Kids_Friendly,Nightlife,Culture,Shopping,Short_stay,Personal_care,Food_markets,Transportation,Leisure,Labels
Neighbourhood,,,,,,,,,,,,,
Agincourt,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.25,4
"Alderwood, Long Branch",0.000000,0.428571,0.142857,0.0,0.142857,0.0,0.142857,0.0,0.0,0.000000,0.0,0.00,4
"Bathurst Manor, Wilson Heights, Downsview North",0.142857,0.142857,0.000000,0.0,0.000000,0.0,0.380952,0.0,0.0,0.095238,0.0,0.00,4
Bayview Village,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,4
"Bedford Park, Lawrence Manor East",0.391304,0.130435,0.000000,0.0,0.130435,0.0,0.260870,0.0,0.0,0.086957,0.0,0.00,4


Next we created the user preferences matrix by calculating the weighted average of the user ratings against the frequency of each category and for all Toronto neighbourhoods in our dataframe. The result was sorted to a top 5 list of recommended neighbourhoods basis user profile which were then plotted on the map of Toronto.

In [73]:
recommendationTable_family = ((Toronto_grouped_new*inputVenues_Family['Rating']).sum(axis=1))/(inputVenues_Family['Rating'].sum())
recommendationTable_family.head(5)

Neighbourhood
Agincourt                                          0.048701
Alderwood, Long Branch                             0.061224
Bathurst Manor, Wilson Heights, Downsview North    0.079159
Bayview Village                                    0.045455
Bedford Park, Lawrence Manor East                  0.087521
dtype: float64

We add the Neighborhood coordinates to map our results:

In [75]:
TOR= recommendationTable_family.to_frame().reset_index()
recommendT=pd.merge(df_toronto, TOR, on="Neighbourhood")
result=recommendT.head(5)
result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,0
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.123377
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.072727
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.082157
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.111317
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.082386


In [77]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighbourhood'].isin(result['Neighbourhood'])]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
45,"Lawrence Manor, Lawrence Heights",Shopping,Personal_care,Health_Fitness,Restaurants,Leisure
61,Parkwoods,Shopping,Kids_Friendly,Leisure,Transportation,Food_markets
62,"Queen's Park, Ontario Provincial Government",Shopping,Restaurants,Kids_Friendly,Fast_Food,Nightlife
63,"Regent Park, Harbourfront",Shopping,Nightlife,Food_markets,Culture,Kids_Friendly
85,Victoria Village,Restaurants,Shopping,Fast_Food,Leisure,Transportation


In [82]:
compare=neighbourhoods_venues_sorted.loc[neighbourhoods_venues_sorted['Neighbourhood'].isin(result['Neighbourhood'])]
compare

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
45,"Lawrence Manor, Lawrence Heights",Shopping,Personal_care,Health_Fitness,Restaurants,Leisure
61,Parkwoods,Shopping,Kids_Friendly,Leisure,Transportation,Food_markets
62,"Queen's Park, Ontario Provincial Government",Shopping,Restaurants,Kids_Friendly,Fast_Food,Nightlife
63,"Regent Park, Harbourfront",Shopping,Nightlife,Food_markets,Culture,Kids_Friendly
85,Victoria Village,Restaurants,Shopping,Fast_Food,Leisure,Transportation


In [78]:
recommendTmap=pd.merge(df_toronto, TOR, on="Neighbourhood")
recommendTmap['Labels']=toronto_merged['Labels']
result_labels=recommendTmap.head(5)
result_labels

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,0,Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.123377,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.072727,4
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.082157,4
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.111317,4
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.082386,4


In [79]:
map_recommended_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(result_labels['Latitude'], result_labels['Longitude'], result_labels['Neighbourhood'], result_labels['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_recommended_neighbourhoods)
       
map_recommended_neighbourhoods